In [1]:
from dict_combine import DictCombine

In [2]:
!pip install pydantic

In [3]:
from pathlib import Path
import requests
import json
from dict_secrets import PONS_SECRET
import pandas as pd
from bs4 import BeautifulSoup
from google_images_download import google_images_download
from stemming.porter2 import stem
import shutil
response = google_images_download.googleimagesdownload() 

def downloadimages(query):
    # keywords is the search query
    # format is the image file format
    # limit is the number of images to be downloaded
    # print urs is to print the image file url
    # size is the image size which can
    # be specified manually ("large, medium, icon")
    # aspect ratio denotes the height width ratio
    # of images to download. ("tall, square, wide, panoramic")
    query = query.split(",")[0]
    arguments = {"keywords": query,
                 "format": "jpg",
                 "limit": 1,
                 "print_urls": False,
                 "size": "medium",}
    
    response.download(arguments)
    image = Path.cwd() / "downloads" / query
    return Path(list(image.glob("*"))[0])

collections_path = Path("/home/artursil/.local/share/Anki2/User 1/collection.media")

ModuleNotFoundError: No module named 'stemming'

In [ ]:
pons_api = "https://api.pons.com/v1/dictionary?l=deen&q={word}"
linguee_api = "https://linguee-api-v2.herokuapp.com/api/v2/translations?query={word}&src=de&dst=en&guess_direction=false&follow_corrections=always"

In [ ]:
reactor_df = pd.read_excel("lln_excel_items_2022-10-6_8170477.xlsx")

In [ ]:
cols_to_keep = ["Item type", "Subtitle", "Translation", "Word", "Lemma", "Word definition", "Part of speech"]
new_cols = ["item_type", "subtitle", "translation", "word", "lemma", "definition", "part_of_speech"]
col_map = {x: y for x, y in zip(cols_to_keep, new_cols)}

In [ ]:
reactor_df = reactor_df[cols_to_keep]
reactor_df.rename(columns=col_map, inplace=True)

In [ ]:
for ix, row in reactor_df.loc[reactor_df["part_of_speech"] == "Noun"].iterrows():
    if row["word"] == "nachlässig":
        break

In [ ]:
for ix, row in reactor_df.iterrows():
    if row["word"] == "nachlässig":
        break

In [ ]:
row

In [ ]:
# lin_entries = json.loads(requests.get(linguee_api.format(word=row.lemma)).text)

In [ ]:
DER_DIE_DAS = {"masculine": "der",
               "neuter": "das",
               "feminine": "die",
               "plural": "die"}

In [ ]:
def switch_to_adj(lin_entries):
    poss = [x["pos"] for x in lin_entries]
    if "verb" not in poss:
        return True
    return False


In [ ]:
def get_lin_entries(word):
    r = requests.get(linguee_api.format(word=word)).text
    if r == "Internal Server Error":
        return "Linguee Error"
    return json.loads(r)

def get_pons_entries(word):
    if r := requests.get(pons_api.format(word=word), 
                         headers={"X-Secret": PONS_SECRET}).text:
        return json.loads(r)
    else:
        return r

# def process_len_entry(l_entries, word, lemma, part_of_speech):
#     entries = []
#     plural = {}
#     for entry in l_entries:
#         entry_text = entry["text"].lower()
#         entry_pos = entry["pos"].split(",")[0].lower().split("/")[0].strip()
#         entry_pos_short = entry_pos[:3]
#         pos = part_of_speech.lower()
#         pos_short = pos[:3]
        
#         if ((entry_text == lemma.lower() or 
#              entry_text == word.lower()) and
#              (entry_pos == pos or entry_pos_short == pos)):
            
#             entries.append(entry)
            
#         if ((entry_text == lemma.lower() or 
#              entry_text == word.lower()) and
#              (entry_pos == "plural"):
            
#             plural = entry
#     return entries, plural


# def get_len_audio(entries, word, collections_path):
#     if not entries:
#         return "No Sound"
#     for entry in entries:
#         audio_link = entry.get("audio_links")
#         if audio_link is not None:
#             audio_link = audio_link[0].get("url")
#         if audio_link:
#             break
#     if audio_link is None:
#         return "No sound"
#     r = requests.get(audio_link)
#     audio_name = f"{word}.mp3"
#     audio_save = f"audio/{word}.mp3"
#     open(audio_save, "wb").write(r.content)
#     shutil.copy(audio_save, collections_path / audio_name)
#     return f"[sound:{audio_name}]"


# def process_adj(row):
#     row_dict = {}                                                                                                                                           
#     word = row.lemma
#     lemma = row.lemma
#     lin_entries = get_lin_entries(word)     
#     pons_entries = get_pons_entries(word)
#     processed_entry = process_len_entry(l_entries=lin_entries, 
#                                         word=word, 
#                                         lemma=word, 
#                                         part_of_speech="adjective")
    
#     audio_save = get_len_audio(processed_entry, word)
#     image = downloadimages(row.definition)
#     row_dict = {
#         "German": word,
#         "Picture": image,
#         "English": row.definition,
#         "Audio": audio_save,
#         "Sample sentence": row.subtitle,
#         "Plural and inflected forms": "",
#         "English Alternatives": row.translation,
#         "Part of speech": "Adjective / Past participle",
#     }
#     return row_dict


# def process_verb(row):
#     row_dict = {}                                                                                                                                           
#     word = row.lemma
#     lemma = row.lemma
#     lin_entries = get_lin_entries(word)
#     if switch_to_adj(lin_entries):
#         return process_adj(row)       
#     pons_entries = get_pons_entries(word)
# #     import pdb; pdb.set_trace()
#     tenses = pons_entries[0]["hits"][0]["roms"][0]["headword_full"].split("&lt;")[-1].split("&gt")[0]
#     processed_entry = process_len_entry(l_entries=lin_entries, word=word, lemma=word, part_of_speech="verb")
#     audio_save = get_len_audio(processed_entry, word)
#     image = downloadimages(row.definition)
#     row_dict = {
#         "German": word,
#         "Picture": image,
#         "English": row.definition,
#         "Audio": audio_save,
#         "Sample sentence": row.subtitle,
#         "Plural and inflected forms": "",
#         "English Alternatives": row.translation,
#         "Part of speech": row.part_of_speech,
#     }
#     return row_dict

# def process_noun(row):
#     row_dict = {}                                                                                                                                           
#     word = row.lemma
#     lemma = row.lemma
#     lin_entries = get_lin_entries(word)
#     if switch_to_adj(lin_entries):
#         return process_adj(row)       
#     pons_entries = get_pons_entries(word)
# #     import pdb; pdb.set_trace()
#     tenses = pons_entries[0]["hits"][0]["roms"][0]["headword_full"].split("&lt;")[-1].split("&gt")[0]
#     processed_entry = process_len_entry(l_entries=lin_entries, word=word, lemma=word, part_of_speech="verb")
#     audio_save = get_len_audio(processed_entry, word)
#     image = downloadimages(row.definition)
#     row_dict = {
#         "German": word,
#         "Picture": image,
#         "English": row.definition,
#         "Audio": audio_save,
#         "Sample sentence": row.subtitle,
#         "Plural and inflected forms": tenses,
#         "English Alternatives": row.translation,
#         "Part of speech": row.part_of_speech,
#     }
#     return row_dict
    

In [ ]:

final_cols = ["German", "Picture", "English", "Audio", "Sample sentence", 
              "Plural and inflected forms", "German Alternatives", 
              "English Alternatives", "Part of speech", "original_word",
              "Source"]

def get_len_audio(entries, word, collections_path):
    if not entries:
        return "No Sound"
    for entry in entries:
        audio_link = entry.get("audio_links")
        if audio_link is not None:
            audio_link = audio_link[0].get("url")
        if audio_link:
            break
    if audio_link is None:
        return "No sound"
    r = requests.get(audio_link)
    audio_name = f"{word}.mp3"
    audio_save = f"audio/{word}.mp3"
    open(audio_save, "wb").write(r.content)
    shutil.copy(audio_save, collections_path / audio_name)
    return f"[sound:{audio_name}]"


def get_examples(l_entry, translation):
    if not l_entry:
        return "", ""
    translations = list(set([translation, stem(translation)]))
    l_entry = l_entry[0]
    translation = translation.split(",")[0]
    found = False
    for translations in translations:
        for trans in l_entry["translations"]:
            if trans["text"] == translation:
                found = True
                break
    if not found or not trans["examples"]:
        for trans in l_entry["translations"]:
            if trans["examples"]:
                break
    examples = trans["examples"]
    if examples:
        return examples[0]["src"], examples[0]["dst"]
    else:
        return "", ""

def process_len_entry(l_entries, word, lemma, part_of_speech, recursive=False):
    if l_entries == "Linguee Error":
        return {}, {}, part_of_speech
    if isinstance(l_entries, dict):
        if "503" in l_entries.get("message", ""):
            print(l_entries.get("message", ""))
            return {}, {}, part_of_speech
    entries = []
    plural = {}
    for entry in l_entries:
        entry_text = entry["text"].lower()
        entry_pos = entry["pos"].split(",")[0].lower().split("/")[0].strip()
        entry_pos_short = entry_pos[:3]
        pos = part_of_speech.lower()
        pos_short = pos[:3]
        if ((entry_text == lemma.lower() or 
             entry_text == word.lower()) and
             (entry_pos == pos or entry_pos_short == pos)):
            
            entries.append(entry)
            
        if ((entry_text == lemma.lower() or 
             entry_text == word.lower()) and
             (entry["pos"].split(",")[-1].lower().strip() == "plural")):
            
            plural = entry
    if not entries and part_of_speech.lower() == "adj" and not recursive:
        return process_len_entry(l_entries, word, lemma, "verb", True)
    if not entries and part_of_speech.lower() == "verb" and not recursive:
        return process_len_entry(l_entries, word, lemma, "adj", True)
    return entries, plural, part_of_speech

def process_pons_entries(pons_entries, pos):
    entries = []
    if not pons_entries:
        return entries
    for entry in pons_entries[0]["hits"]:
        if roms := entry.get("roms", []):
            if pos in roms[0].get("wordclass", ""):
                entries.append(entry)
    return entries

class LangReactorEntry():
    def __init__(self, row: pd.Series,
                 collections_path=Path("/home/artursil/.local/share/Anki2/User 1/collection.media")):       
        self.row = row
        self.word = row.word
        self.lemma = row.lemma
#         self.definition = row.definition
        self.subtitle = row.subtitle
        self.translation = row.translation
        self.pos = row.part_of_speech
        self.lin_entries = get_lin_entries(self.lemma)
        self.pons_entries = get_pons_entries(self.lemma)
        self.collections_path = collections_path

    @property
    def definition(self):
        if pd.isnull(self.row.definition):
            spcial_char_map = {ord('ä'):'ae', ord('ü'):'ue', ord('ö'):'oe', ord('ß'):'ss'}
            definition = self.lemma.translate(spcial_char_map)
            print(definition)
            return definition
        return self.row.definition
        
        
    def __process_entries(self):
        ple = process_len_entry(l_entries=self.lin_entries, 
                                word=self.word, 
                                lemma=self.lemma, 
                                part_of_speech=self.pos)
        
        self.processed_entries, self.plural, self.pos = ple
        
    @property
    def tenses_plural(self):
        if self.pos.lower() == "verb":
            return self.__get_tenses()
        if self.pos.lower() == "noun":
            return self.__get_plural()
        return ""
    
    def __get_tenses(self):
        
        head = self.pons_entries[0]["roms"][0]["headword_full"]
        soup = BeautifulSoup(head)
#         import pdb; pdb.set_trace()
        return soup.find("span").get("title")
    
    @property
    def pons_plural(self):
        if not self.pons_entries:
            return ""
        head = self.pons_entries[0]["roms"][0]["headword_full"]
        soup = BeautifulSoup(head)
        
        if flexicon := soup.find("span", {"class": "flexion"}):
            text = flexicon.text
        else:
            text = ""
        word = self.lemma.capitalize() + text.split("-")[-1].split(">")[0]
#         import pdb; pdb.set_trace()
        return f"die {word}"
    
    def __get_plural(self):
        if self.plural:
            return f"die {self.plural['text'].capitalize()}"
        else:
            return self.pons_plural
    
    @property
    def example_src(self):
        src, _ = get_examples(self.processed_entries, 
                              self.definition)
        if src:
            return f"1. {self.subtitle}<br> 2. {src}"
        else:
            return self.subtitle
    
    @property
    def example_translation(self):
        _, dst = get_examples(self.processed_entries, 
                              self.definition)
        if dst:
            return f"1. {self.translation}<br> 2. {dst}"
        else:
            return self.translation
    
    @property
    def processed_word(self):
        if self.pos.lower() != "noun":
            return self.lemma
        elif not self.processed_entries:
            return self.lemma.capitalize()
        ddd = self.processed_entries[0]["pos"].split(",")[-1].strip()
        return f"{DER_DIE_DAS[ddd]} {self.lemma.capitalize()}"
    
    @property
    def get_image(self):
        image_path = downloadimages(self.definition)
        img_name = image_path.parent.stem + image_path.suffix
        img_dst = self.collections_path / img_name
        shutil.copy(image_path, img_dst)
        return f"<img src='{img_name}'>"

    
    def process_row(self):
        self.__process_entries()
        self.pons_entries = process_pons_entries(self.pons_entries,
                                                 self.pos.lower())
        audio_save = get_len_audio(self.processed_entries, self.word, self.collections_path)
        row_dict = {
            "German": self.processed_word,
            "Picture": self.get_image,
            "English": self.definition,
            "Audio": audio_save,
            "Sample sentence": self.example_src,
            "Plural and inflected forms": self.tenses_plural,
            "German Alternatives": "",
            "English Alternatives": self.example_translation,
            "Part of speech": self.pos,
            "original_word": self.word,
            "Source": "Language Reactor"
        }
        return row_dict
    
    def __call__(self):
        if self.lin_entries == {'message': 'The Linguee server returned 503'}:
            return "503 Error"
        return self.process_row()
        

In [ ]:
# lang_reactor = LangReactorEntry(row)


In [ ]:
# entry = lang_reactor()

In [ ]:
# entry

In [ ]:
import time

In [ ]:
while True:
    try:
        ready_for_anki = pd.read_csv("ready_for_anki.csv", names=final_cols, header=None)
    except FileNotFoundError:
        ready_for_anki = pd.DataFrame()
    ready_entries = []
    for ix, row in reactor_df.loc[reactor_df.item_type == "Word"].iterrows():
        print(ix)
        print(row.word)
        if ix <= 278:
            continue
        if not ready_for_anki.empty:
            if row.word in ready_for_anki.original_word.to_list():
                continue
        ready_entry = LangReactorEntry(row)()
    #     import pdb; pdb.set_trace()
        if ready_entry == "503 Error":
            print(ready_entry)
            break
        ready_entries.append(ready_entry)
        time.sleep(10)
    if ready_entry == "503 Error":
        time.sleep(30*60)
    df_tmp = pd.DataFrame(ready_entries)
    ready_for_anki = ready_for_anki.append(df_tmp)
    ready_for_anki.to_csv("ready_for_anki.csv", index=False)"
#     import pdb; pdb.set_trace()   

In [ ]:
df_tmp = pd.DataFrame(ready_entries)
ready_for_anki = ready_for_anki.append(df_tmp)
ready_for_anki.to_csv("ready_for_anki.csv", index=False)

In [ ]:
%debug

In [ ]:
ready_for_anki = pd.read_csv("ready_for_anki.csv", header=None)

In [ ]:
ready_for_anki = pd.read_csv("ready_for_anki.csv", names=final_cols, header=None)
ready_for_anki.head(1)

In [ ]:
ready_for_anki["Audio"] = ["".join(x.split("audio/")) for x in ready_for_anki["Audio"]]
ready_for_anki["Picture"] = [f"<img src='{x}'>" for x in ready_for_anki["Picture"]]


In [ ]:
ready_for_anki.head(1)

In [ ]:
# for ix, row in ready_for_anki.iterrows():
#     audio = row.Audio

 
ready_for_anki.reset_index(inplace=True)
for ix, row in ready_for_anki.iterrows():
    pic = row.Picture
    if pic.lower() != "no sound":
        pic = pic.split("<img src='")[-1].split("'>")[0]
        print(pic)
        pic_name = f"{pic.split('/')[-2]}.{pic.split('.')[-1]}"
        print(pic_name)

        shutil.copy(pic, collections_path / pic_name) 
        ready_for_anki.loc[ix, "Picture"] = f"<img src='{pic_name}'>"

In [ ]:
for ix, row in ready_for_anki.iterrows():
    pic = row.Picture
    if pic.lower() != "no sound":
        pic = pic.split("<img src='")[-1].split("'>")[0]
        print(pic)
        pic_name = f"{pic.split('/')[-2]}.{pic.split('.')[-1]}"
        print(pic_name)

        shutil.copy(pic, collections_path / pic_name)

In [ ]:
ready_for_anki

In [ ]:
ready_for_anki = ready_for_anki.loc[~pd.isnull(ready_for_anki["Sample sentence"])]

ready_for_anki.to_csv("ready_for_anki2.csv", index=False, header=None)

In [ ]:
 <img src=”duck.jpg”>
    <source src="horse.mp3" type="audio/mpeg">